In [1]:

import pandas as pd
import numpy as np

# для работы с адресами
from geopy.geocoders import Nominatim
#для работы с системными файлами
import os
#для работы с регулярными выражениями
import re
#для работы со временем
import time

### Сбор данных

___

**Так как данные поступали в течени года и можно было с ними начать работать с самого начала проекта, было создано несколько функций для сбора всей информации в один датасэт**

In [2]:
DIRECTORY_fold = 'data/'

def get_list_all_files() -> list:
    """
    Returns:
        list: функция возвращает названия всех xlsx документов в нужном формате из папки data
    """
    
    ALL_FILES = []
    folds = os.listdir(DIRECTORY_fold)
    for fold in folds:
        DIRECTORY_file = DIRECTORY_fold + str(fold)
        files = os.listdir(DIRECTORY_file)
        files = os.listdir(DIRECTORY_file)
        ALL_FILES.extend(files)
    
    return ALL_FILES


def conversion(df: pd.DataFrame) -> pd.DataFrame:
    """Функция преобразует датасэт под единный формат и возвращает его"""
    
    if len(df.columns) > 10:
        raise TypeError('Не подходящий формат')
    else:
        cols = df.columns[:8]
        df = df[cols]
        df['Unnamed: 3'] = df['Unnamed: 3'].apply(lambda x: np.nan if x == 'Наименование' else x)
        df.drop(df.columns[:2], axis=1, inplace=True)
        df.dropna(axis=0, inplace=True)
        df.set_index(np.arange(len(df.index)), inplace=True)

        #переименовываем столбцы
        df.rename(columns={'Unnamed: 2':'num', 'Unnamed: 3':'ration', 'Unnamed: 4':'address',
                        'Unnamed: 5':'start', 'Unnamed: 6':'finish',
                        'Unnamed: 7':'comment'}, inplace=True)

        # функция которая сокращает название рациона
        def edit_ration_name(name):
            if 'Снижение' in name:
                return name.replace('Снижение', '').strip()
            elif 'Баланс' in name:
                return name.replace('Баланс', '').strip()
            elif 'Classic' in name:
                return name.replace('Classic', '').strip()
            elif 'Набор' in name:
                return name.replace('Набор', '').strip()
            

        df['start'] = df['start'].apply(lambda x: int(x[0]) if len(x) <= 4 else int(x[:2])) # заменяем начало интервала со строкого формата на числовой
        df['finish'] = df['finish'].apply(lambda x: int(x[0]) if len(x) <= 4 else int(x[:2]))# заменяем конец интервала со строкого формата на числовой
        df['ration'] = df['ration'].apply(edit_ration_name)
    
        return df


def get_df(files: list) -> pd.DataFrame:
    """
    Args:
        files (list): Список названий файлов

    Returns:
        pd.DataFrame: датасэт из всех xlsx файлов в формате DataFrame
    """
    for i, file in enumerate(files):
        try:
            month = int(file.split('.')[1])
        except ValueError:
            print(file)
        if i == 0:
            data = pd.read_excel(f'data/{month}/{file}')
            data = conversion(data)
            data['date'] = [file.split('.')[:2]] * len(data)
            lenght = len(data)
        else:
            try:
                data_plus = pd.read_excel(f'data/{month}/{file}')
                data_plus = conversion(data_plus)
                data_plus['date'] = [file.split('.')[:2]] * len(data_plus)
                lenght += len(data_plus)
                data = pd.concat([data, data_plus], axis=0)
            except Exception:
                continue
    
    #data = data.drop(['Unnamed: 9'], axis=1)
    print(f'data shape is {data.shape[0]} should equal {lenght}')

    return data

In [3]:
ALL_FILES = get_list_all_files()
df = get_df(ALL_FILES)

/tmp/ipykernel_34912/499859534.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Unnamed: 3'] = df['Unnamed: 3'].apply(lambda x: np.nan if x == 'Наименование' else x)
/tmp/ipykernel_34912/499859534.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df.columns[:2], axis=1, inplace=True)
/tmp/ipykernel_34912/499859534.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

data shape is 162894 should equal 162894


### Преоброзование датасэта

___

In [4]:
df_test = df.copy(deep=True)

In [5]:
# Признак date преобразуем в формат datetime
df_test['date'] = df_test['date'].apply(lambda x: '/'.join(x) + '/2022')
df_test['date'] = pd.to_datetime(df_test['date'], format='%d/%m/%Y')

# Выделяем отдельные признаки месяца, дня месяца и дня недели
df_test['month'] = df_test['date'].dt.month
df_test['day'] = df_test['date'].dt.day
df_test['dow'] = df_test['date'].dt.day_of_week

In [6]:
# В ноябре названия некоторых пакетов поменялись, но сущность осталось таже. Поэтому переименовываем их обратно в старые, чтобы лучше ориентироваться
ration_new_dict = {
    '5 блюд 3 дня': 'M',
    '6 блюд 3 дня': 'L',
    '4 блюда 3 дня': 'S',
    '3 блюда 3 дня': 'XS',
    '5 блюд 2 дня': 'M (2)',
    '4 блюда 2 дня': 'S (2)',
    '6 блюд 2 дня': 'L (2)',
    '3 блюда 2 дня': 'XS (2)'
    }

df_test['ration'] = df_test['ration'].apply(lambda x: ration_new_dict[x] if x in ration_new_dict.keys() else x)

In [7]:
# удаляем ненужные признаки
df_test = df_test.drop(['num', 'comment'], axis=1)
df_test.reset_index(inplace=True, drop=True)

In [8]:
def get_location(df: pd.DataFrame) -> list:
    """ Функция возвращает короткий адрес
        для улучшения поиска в библиотеке geopy

    Args:
        df (pd.DataFrame): Датафрейм с полными адресами включая квартиру, этаж и т.д

    Returns:
        list: Возвращает два списка: 
        1. C улицей и номером дома
        2. Населенный пункт
    """
    # список адресов
    list_of_address = df['address'].to_list()

    # созадем списки улицы, номера дома и города
    street_list, area_ekb_final = [], []
    
    # достаем названия улиц
    for elem in list_of_address:
        dig = re.findall(r'\d+', elem)
        ls = elem.split(',')
        if dig != []:
            result = [' '.join(ls[ind-1: ind + 1]) for ind, part in enumerate(ls) if dig[0] in part]
        else:
            result = 'not_detected'
        street_list.append(result)
    
    for ind, elem in enumerate(street_list):
        if elem == '':
            result = elem.split(',')[1:3]
            street_list[ind] = ' '.join(result)
    
    street_list = [elem[1].strip() if elem[0] == ' ' else elem[0].strip() for elem in street_list]
    street_list = [elem.replace('улица', 'ул').replace('проспект', 'пр-кт') for elem in street_list]
    
    #----------------------------------------------------------------
    # Достаем из полного адреса населенный пункт

    # Список крупных населенных пунктов
    area_ekb = [
    'Екатеринбург','Верхняя Пышма', 'Арамиль', 'Березовский', 'Ревда',
    'Среднеуральск', 'Полевской', 'Берёзовский', 'Заречный', 'Сысерть'
    ]

    # Проходимся по каждому адресу и достаем населенный пункт.
    # Сначала идет проверка на крупные наеленные пункты,
    # затем остается проверка на небольшие деревни и села.
    for elem in list_of_address:
        elem_list = [val.strip() for val in elem.split(',')]
        elem_set = set(elem_list)
        result = elem_set.intersection(area_ekb)
        if result == set():
            result = re.findall(r'снт \w+,', elem.lower())
            if result == []:
                result = re.findall(r'поселок \w+,', elem.lower())
                if result == []:
                    result = re.findall(r'село \w+,', elem.lower())
                    if result == []:
                        result ='Екатеринбург'
                    
        area_ekb_final.append(''.join(result))


    return street_list, area_ekb_final

In [9]:
street_list, area_ekb_exp = get_location(df_test)

df_test['street'] = street_list
df_test['area'] = area_ekb_exp
df_test['short_address'] = df_test['area'] + ', ' + df_test['street']

In [10]:
df_test

,ration,address,start,finish,date,month,day,dow,street,area,short_address
0,1500,"Россия, Свердловская область, Екатеринбург, Ор...",6,8,2022-03-05,3,5,5,ул Старых Большевиков 4,Екатеринбург,"Екатеринбург, ул Старых Большевиков 4"
1,1500,"Таганская улица, 91, микрорайон Эльмаш, Орджон...",6,8,2022-03-05,3,5,5,Таганская ул 91,Екатеринбург,"Екатеринбург, Таганская ул 91"
2,1000,"улица Кривоусова, 18Д, Верхняя Пышма, Свердлов...",6,8,2022-03-05,3,5,5,ул Кривоусова 18Д,Верхняя Пышма,"Верхняя Пышма, ул Кривоусова 18Д"
3,L,"улица Машиностроителей, 6Б, Верхняя Пышма, Све...",6,8,2022-03-05,3,5,5,ул Машиностроителей 6Б,Верхняя Пышма,"Верхняя Пышма, ул Машиностроителей 6Б"
4,2000,"Россия, Свердловская область, городской округ ...",6,8,2022-03-05,3,5,5,Счастливая ул 41/6,Екатеринбург,"Екатеринбург, Счастливая ул 41/6"
...,...,...,...,...,...,...,...,...,...,...,...
162889,750,"Донбасская улица, 1, микрорайон Уралмаш, Екате...",9,11,2022-11-07,11,7,0,Донбасская ул 1,Екатеринбург,"Екатеринбург, Донбасская ул 1"
162890,1500,"Екатеринбург, ул Хмелева, 10 , кв. 70, эт. 5, ...",10,12,2022-11-07,11,7,0,ул Хмелева 10,Екатеринбург,"Екатеринбург, ул Хмелева 10"
162891,2000,"Екатеринбург, ул Калинина, 59 , кв. 339, эт. 4...",10,12,2022-11-07,11,7,0,ул Калинина 59,Екатеринбург,"Екатеринбург, ул Калинина 59"
162892,1000,"Россия, Свердловская область, Екатеринбург, Ор...",10,12,2022-11-07,11,7,0,Орджоникидзевский район Зелёный переулок 2ж,Екатеринбург,"Екатеринбург, Орджоникидзевский район Зелёный..."


**Каждому адресу присваеваем широту и долготу**

Чтобы присвоить каждому адресу координаты, я решил использовать библиотеку *geopy*. Но если напрямую использовать данную библиотеку через метод *apply*, то разметка геоданных займет около 18 часов. Это связанно из-за невозможности распараллеливать вычисления(разрешен 1 запрос в n-время) и из-за траты времени на обработку запроса в диапазоне от 1 до 2 секунд. 

Поэтому было принято решение создать словарь, где ключ был бы равен какому-то адресу, а значение - это геоданные данного адреса. Так обработанный запрос записывался бы в словарь и уже при новом запросе с похожим адресом обращался бы к нему.

In [11]:
# Данный словарь уже у нас записан, поэтому не придется создавать его заново

#Создаем объект класса для Nominatim из библиотеки geopy
geolocator = Nominatim(user_agent='location')
#Загружаем наш словарь(он уже заполнен некторыми адресами)
dict_loc= np.load('secondary_data/dict_loc.npy', allow_pickle=True).item()
#Геоданные Екатеринбурга. Если geopy не распознает наш адрес, то присвоем данному адресу координаты Екатеринбурга
ekb_loc = [56.838011, 60.597474]

def get_geolocation(st: str) -> list:
    """Функция возвращает координаты входного адреса

    Args:
        st (str): Адрес дома

    Returns:
        list: Список координат 
    """
    try:
        result = dict_loc[st]
    # Если нет данного адреса в словаре, то срабатывает исключение
    except KeyError:
        try:
            location = geolocator.geocode(st, timeout=None)
            dict_loc.setdefault(st, [location.latitude, location.longitude])
            result = dict_loc[st]
        # Если geocode не распознал адрес,
        # то срабатывает исключение и забиваются координаты Екатеринбурга
        except AttributeError:
            dict_loc.setdefault(st, ekb_loc)
            result = ekb_loc 
    
    return result

In [12]:
# Поставить True, если нужно запустить ячейку
# Данная ячейка дополняет наш словарь новыми адресами,
# Она была использована для первичного заполнения словаря, когда он был пуст
flag = True

if flag is True:
    # Показывает количесвто уникальных адресов с каждой итерацией
    dict_density = []

    # Условие при котором можно остановить цикл поиска уникальных ключей,
    # достаточное для разметки данных в адекватный промежуток времени
    CONDITION = df_test['short_address'].nunique() * 0.8

    # Для быстрого поиска уникальных адресов, решено разбить датасэт на выборки из 10 частей
    split_data = np.array_split(df_test, 10)

    
    while len(dict_loc) <= CONDITION:
        # Проход по всем выборкам происходит со смещением в середину.
        # Это сделано для того, чтобы как можно быстрее охватить новые адреса и включить их в словарь,
        # так как за год новые адреса(клиенты) появлялись постоянно, а старые могли уходить.
        for i in [0, 9, 1, 8, 2, 7, 3, 6, 4, 5]:
            # Чтобы не прочитывать всю выборку, решено взять лишь часть в размере 33% 
            var_data = split_data[i].sample(3000)
            var_data['loc'] = var_data['short_address'].apply(get_geolocation)
            dict_density.append(len(dict_loc))
        
        #Сохраняем наш словарь
        np.save('secondary_data/dict_loc.npy', dict_loc)
        print(f'Уникальных объектов: {dict_density[-1]}, За цикл прибавилось: {max(dict_density) - min(dict_density)}')

In [13]:
start = time.time()
df_test['location'] = df_test['short_address'].apply(get_geolocation)
end = time.time()
print(f'Обработка адресов заняло {int((end - start) / 60)} минут(-ы)')

Обработка адресов заняло 3 минут(-ы)


**В итоге вместо 18 часов, добавление координат занимает несколько минут**

In [14]:
location_list = df_test['location'].to_list()
df_test['latitude'] = [elem[0] for elem in location_list]
df_test['longitude'] = [elem[1] for elem in location_list]

**Можно привязать погоду к каждой дате, чтобы увидеть в будущем какие либо закономерности**

In [15]:
weather = pd.read_csv('secondary_data/weather.csv', sep=';')
weather.head(3)

,Местное время в Екатеринбурге,T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
31.12.2022 23:00,-24.2,747.2,776.5,-0.3,73,"Ветер, дующий с востока",1,NaN,NaN,70 – 80%.,...,"Перистых, перисто-кучевых или перисто-слоистых...",10.0,-27.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31.12.2022 20:00,-23.2,747.5,776.6,0.0,73,"Штиль, безветрие",0,NaN,NaN,70 – 80%.,...,"Перистые нитевидные, иногда когтевидные, не ра...",10.0,-26.7,Следы осадков,12.0,NaN,NaN,NaN,NaN,NaN
31.12.2022 17:00,-22.2,747.5,776.5,0.1,75,"Штиль, безветрие",0,NaN,NaN,60%.,...,Перистые (часто в виде полос) и перисто-слоист...,10.0,-25.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Нам нужна только температура из данной таблицы
date, forecast = weather['Местное время в Екатеринбурге'].index, weather['Местное время в Екатеринбурге'].values
weather = pd.DataFrame({'date': date, 'weather': forecast})
weather['date'] = pd.to_datetime(weather['date']).dt.date

# Групируем данные по дате и находим среднюю температуру
weather = weather.groupby('date', as_index=False).mean()
weather['date'] = pd.to_datetime(weather['date'])

In [17]:
# Добавляем нашу температуру к главной таблице
df_test = df_test.merge(weather, how='left')
df_test['weather'] = df_test['weather'].apply(int)
df_test.head()

,ration,address,start,finish,date,month,day,dow,street,area,short_address,location,latitude,longitude,weather
0,1500,"Россия, Свердловская область, Екатеринбург, Ор...",6,8,2022-03-05,3,5,5,ул Старых Большевиков 4,Екатеринбург,"Екатеринбург, ул Старых Большевиков 4","[56.88796345, 60.63480718496369]",56.887963,60.634807,9
1,1500,"Таганская улица, 91, микрорайон Эльмаш, Орджон...",6,8,2022-03-05,3,5,5,Таганская ул 91,Екатеринбург,"Екатеринбург, Таганская ул 91","[56.9079089, 60.625762968603304]",56.907909,60.625763,9
2,1000,"улица Кривоусова, 18Д, Верхняя Пышма, Свердлов...",6,8,2022-03-05,3,5,5,ул Кривоусова 18Д,Верхняя Пышма,"Верхняя Пышма, ул Кривоусова 18Д","[56.9730964, 60.5771942]",56.973096,60.577194,9
3,L,"улица Машиностроителей, 6Б, Верхняя Пышма, Све...",6,8,2022-03-05,3,5,5,ул Машиностроителей 6Б,Верхняя Пышма,"Верхняя Пышма, ул Машиностроителей 6Б","[56.9838735, 60.55081838138887]",56.983874,60.550818,9
4,2000,"Россия, Свердловская область, городской округ ...",6,8,2022-03-05,3,5,5,Счастливая ул 41/6,Екатеринбург,"Екатеринбург, Счастливая ул 41/6","[56.747284, 60.5636198]",56.747284,60.563620,9


### Работа с пропусками

In [18]:
df_test.isnull().sum()

ration           1580
address             0
start               0
finish              0
date                0
month               0
day                 0
dow                 0
street              0
area                0
short_address       0
location            0
latitude            0
longitude           0
weather             0
dtype: int64

**В признаке *ration* отсутствуют данные для 1000 строк, заполним их.** 

**Также создадим несколько дополнительных признаков - это цена за каждый пакет и строковое значение месяца**

In [19]:
price_list ={
    '1500': 650, '1000': 600, '2000': 700, '750': 550, '2500': 770,
    'M': 610, 'L': 670, 'S': 550, '3500': 850, 'XS': 500, 'M (2)': 320, 'L (2)': 330,
    '1500 (1)': 330, '1000 (1)': 300, 'S (2)': 360, '2000 (1)': 350,
    '750 (1)': 280, 'XS (2)': 330, '2500 (1)': 390, '1200': 630,
    '3500 (1)': 430, '1800': 680, '2400': 750
    }

month_dict = {
    1:'Январь', 2:'Февраль', 3:'Март', 4:'Апрель',
    5:'Май', 6:'Июнь', 7:'Июль', 8:'Август',
    9:'Сентябрь', 10:'Октябрь', 11:'Ноябрь', 12:'Декабрь'
    }

In [20]:
# Заполняем пропуски
ration_list = list(price_list.keys())
df_test['ration'] = df_test['ration'].apply(lambda x: np.random.choice(ration_list) if x == None else x)

# Пропусков больше нет
df_test.isnull().sum()

ration           0
address          0
start            0
finish           0
date             0
month            0
day              0
dow              0
street           0
area             0
short_address    0
location         0
latitude         0
longitude        0
weather          0
dtype: int64

In [21]:
# В ноябре появились новые названия пакетов с пометкой "зеленая", заменяем их на уже привычные названия, так как суть их не поменялась.
df_test['ration'] = df_test['ration'].apply(lambda x:x.replace('зеленая', '').strip() if 'зеленая' in x else x)
df_test['ration'] = df_test['ration'].apply(lambda x:'1500 (1)' if 'Меню' in x else x)

In [22]:
# Создаем признаки цены и строкого значения месяца
df_test['price'] = df_test['ration'].map(price_list)
df_test['month_str'] = df_test['month'].map(month_dict)

In [23]:
# Отсартируем таблицу по дате
df = df_test.sort_values(by='date').reset_index(drop=True)

In [24]:
# создадим признак переодичности покупок клиента
df['period'] = df.groupby('address', as_index=False)['date'].diff()
df['period'] = df['period'].dt.days
df['period'].fillna(-1, inplace=True)
df['period'] = df['period'] + 1

In [25]:
df.head(5)

,ration,address,start,finish,date,month,day,dow,street,area,short_address,location,latitude,longitude,weather,price,month_str,period
0,750,"Екатеринбург, ул Вилонова, 22 , кв. 77, эт. 10...",7,9,2022-01-02,1,2,6,ул Вилонова 22,Екатеринбург,"Екатеринбург, ул Вилонова 22","[56.864781199999996, 60.64482563954775]",56.864781,60.644826,-5,550,Январь,0.0
1,1500,"Екатеринбург, ул Михеева М.Н., 2 , кв. 793, эт...",6,8,2022-01-02,1,2,6,ул Михеева М.Н. 2,Екатеринбург,"Екатеринбург, ул Михеева М.Н. 2","[56.7805477, 60.5514291]",56.780548,60.551429,-5,650,Январь,0.0
2,1500,"Предельная улица, 15, посёлок Совхозный, муниц...",6,8,2022-01-02,1,2,6,Предельная ул 15,Екатеринбург,"Екатеринбург, Предельная ул 15","[56.7631478, 60.5691115]",56.763148,60.569111,-5,650,Январь,0.0
3,L (2),"Россия, Свердловская область, Полевской городс...",6,8,2022-01-02,1,2,6,микрорайон Солнечный 119,"село курганово,","село курганово,, микрорайон Солнечный 119","[56.838011, 60.597474]",56.838011,60.597474,-5,330,Январь,0.0
4,L (2),"микрорайон Солнечный, 119, село Курганово, Пол...",6,8,2022-01-02,1,2,6,микрорайон Солнечный 119,"село курганово,","село курганово,, микрорайон Солнечный 119","[56.838011, 60.597474]",56.838011,60.597474,-5,330,Январь,0.0


___

**Также для простоты анализа и кластеризации, создадим новую таблицу характеризующую каждого клиента**

**Так как у нас нет id каждого клиента, нам нужно будет создать их самим. Id будет создаваться на основе уникального адреса**

In [26]:
# список уникальных адресов
ls_clietns = df['address'].unique()

id_ls = []
# Создаем словарь в котором каждому уникальному адресу присваивается свой id,
# но так как словарь уже был создан, то просто загружаем его
id_dict = {}
#id_dict = np.load('secondary_data/dict_loc.npy')

# функция генерирует id
def gen_id():
    return np.random.choice(['j', 'k', 's', 'x']) + str(np.random.randint(10000, 99999))

# проверем нет ли уже такого же созданного id и присваиваем каждому уникальному адресу свой
for client in ls_clietns:
    
    id_ = gen_id()
    while id_ in id_ls:
        id_ = gen_id()
    
    id_ls.append(id_)
    id_dict[client] = id_dict.setdefault(client, id_)

df['id'] = df['address'].map(id_dict) 

In [27]:
# сохраняем наш словарь с id
np.save('secondary_data/id_dict.npy', id_dict)

In [28]:
# создаем таблицу клиентов
clients = df.groupby('id', as_index=False)['address'].count()

In [29]:
clients.drop('address', axis=1, inplace=True)
clients = clients.merge(df[['id','short_address', 'latitude','longitude']])
clients.drop_duplicates(inplace=True)

In [30]:
# закодируем интервал доставки
df['time_zone'] = df['start'].map({6:1, 7:2, 8:3, 9:4, 10:5, 11:6})

In [31]:
group_dict = {
    'weather': 'count',
    'period': 'median',
    'ration': 'nunique',
    'price': 'sum',
    'time_zone': 'median'
}

In [32]:
groups = df.groupby('id', as_index=False).agg(group_dict)
clients = clients.merge(groups)
clients = clients.rename(columns={
    'weather':'sum_trans',
    'ration':'nuniq_ration',
    'price':'revenue'
    })

In [33]:
pop_ration = df.groupby('id', as_index=False)['ration'].agg(pd.Series.mode)
pop_ration['ration'] = pop_ration['ration'].apply(lambda x: x[1] if type(x) is np.ndarray else x)
clients = clients.merge(pop_ration)
clients = clients.rename(columns={'ration':'pop_ration'})

In [34]:
print(f'{clients.shape[0]} уникальных клиентов')
clients.head(5)

14106 уникальных клиентов


,id,short_address,latitude,longitude,sum_trans,period,nuniq_ration,revenue,time_zone,pop_ration
0,j10039,"Екатеринбург, Походная ул 74",56.774542,60.634243,10,3.5,1,6500,4.0,1500
1,j10138,"Екатеринбург, ул Ленинградская 16",56.819255,60.544925,3,3.0,1,1650,3.0,750
2,j10182,"Екатеринбург, ул Куйбышева 139",56.831166,60.639862,2,2.0,1,1100,3.0,S
3,j10254,"Екатеринбург, Пролетарская ул 47А",56.843389,60.606278,1,0.0,1,650,5.0,1500
4,j10276,"Екатеринбург, ул Александра Матросова 1",56.838011,60.597474,3,10.0,2,1720,2.0,M


**Создадим отдельную таблицу для работы с временным рядом**

In [35]:
time_series = df.groupby('date', as_index=False)['ration'].count()# группировка количества заказов по дням
time_series = time_series.rename(columns={'ration': 'count'})

**Сохраняем готовые таблицы для дальнейшей работы**

In [36]:
df.to_csv('support/transactions.csv', index=False)
clients.to_csv('support/clients.csv', index=False)
time_series.to_csv('support/time_series.csv', index=False)